In [2]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)

### Rotation convention
The roll, pitch, yaw (RPY) convention common in aerospace follows an intrinsic (that is, about the moving coordinate system) rotation about the body Z, Y, and X axes. A confusing aspect is that the order of rotations is inverted w.r.t. the naming since the yaw rotation is the first one. The naming refers to the conventional ordering of the rotations as states, which is roll first, then pitch, then yaw.

For the calculation of the associated rotation matrices see the code below. This also shows that the RPY convention is equal to an extrinsic (that is, about the inertial coordinate system) rotation in the order XYZ.

The convention used here thus corresponds to the Euler ZYX mentioned in this source: http://web.mit.edu/2.05/www/Handout/HO2.PDF \
And this corresponds to the rotation matrix derived here: https://link.springer.com/article/10.1007/s11071-022-08212-w

In [ ]:
L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x, y, z, roll, pitch, yaw, q1, q2, q3 = sp.symbols('x_b y_b z_b roll pitch yaw q_1 q_2 q_3', real=True)

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

In [4]:
# ROTATIONS
# Intrinsic ZYX
Rz = sp.Matrix([
    [sp.cos(yaw), -sp.sin(yaw), 0],
    [sp.sin(yaw),  sp.cos(yaw), 0],
    [0,            0,           1]
])

Ry = sp.Matrix([
    [sp.cos(pitch),  0, sp.sin(pitch)],
    [0,              1, 0            ],
    [-sp.sin(pitch), 0, sp.cos(pitch)]
])

Rx = sp.Matrix([
    [1, 0,            0           ],
    [0, sp.cos(roll), -sp.sin(roll)],
    [0, sp.sin(roll),  sp.cos(roll)]
])

# Intrinsic ZYX: Postmultiplication
R_zyx_intrinsic = Rz * Ry * Rx

# Extrinsic XYZ: Premultiplication
R_xyz_extrinsic = Rz * Ry * Rx

sp.pprint(R_xyz_extrinsic)
sp.pprint(R_zyx_intrinsic)

⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)                     sin(roll)⋅cos(pitch)                      ↪

↪ pitch)⋅cos(roll)⋅cos(yaw) + sin(roll)⋅sin(yaw)⎤
↪                                               ⎥
↪ pitch)⋅sin(yaw)⋅cos(roll) - sin(roll)⋅cos(yaw)⎥
↪                                               ⎥
↪            cos(pitch)⋅cos(roll)               ⎦
⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)    

### Conventions
For arm 1, the nominal configuration is straight out to the right of the body (i.e. along positive $Y_{b}$). For arm 2, the nominal configuration is straight out to the left of the body (i.e. along the negative $Y_{b}$).

Rotations around relative axes are intrinsic and thus you use postmultiplication of the rotation.

In [21]:
# Body frame b: FRD at vehicle CoM
T_Ib = sp.Matrix([[sp.cos(yaw)*sp.cos(pitch), sp.cos(yaw)*sp.sin(pitch)*sp.sin(roll)-sp.sin(yaw)*sp.cos(roll), sp.cos(yaw)*sp.sin(pitch)*sp.cos(roll)+sp.sin(yaw)*sp.sin(roll), x],
                  [sp.sin(yaw)*sp.cos(pitch), sp.sin(yaw)*sp.sin(pitch)*sp.sin(roll)+sp.cos(yaw)*sp.cos(roll), sp.sin(yaw)*sp.sin(pitch)*sp.cos(roll)-sp.cos(yaw)*sp.sin(roll), y],
                  [-sp.sin(pitch), sp.cos(pitch)*sp.sin(roll), sp.cos(pitch)*sp.cos(roll), z],
                  [0, 0, 0, 1]])

# Frame 0: Z axis of rotation pointing forward, X axis pointing up, Y axis pointing right (90 deg rotation around Yb)
T_b0 = sp.Matrix([[0, 0, 1, 0],
                  [0, 1, 0, 0],
                  [-1, 0, 0, 0],
                  [0, 0, 0, 1]])

# Frame 1: Rotation around Z0 by q1
T_01 = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0, 0],
                [sp.sin(q1),  sp.cos(q1), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

# Frame 2: Translation along Y1 of L1, then rotation around Y1 by pi/2 and rotation around new Z2 by q2
rot_y1 = sp.Matrix([[sp.cos(sp.pi/2), 0, sp.sin(sp.pi/2)],
                    [0, 1, 0],
                    [-sp.sin(sp.pi/2), 0, sp.cos(sp.pi/2)]])
rot_z2 = sp.Matrix([[sp.cos(q2), -sp.sin(q2), 0],
                    [sp.sin(q2), sp.cos(q2), 0],
                    [0, 0, 1]])
rot_y1z2 = rot_y1 * rot_z2 # postmultiply because z2 is relative axis
T_12 = sp.diag(rot_y1z2, 1)
T_12[0, 3] = L1

# Frame 3: Translation along Y2 by L2, rotation about Y2 by -pi/2, then rotation about new Z3 by q3
rot_y2 = sp.Matrix([[sp.cos(-sp.pi/2), 0, sp.sin(-sp.pi/2)],
                    [0, 1, 0],
                    [-sp.sin(-sp.pi/2), 0, sp.cos(-sp.pi/2)]])
rot_z3 = sp.Matrix([[sp.cos(q3), -sp.sin(q3), 0],
                    [sp.sin(q3), sp.cos(q3), 0],
                    [0, 0, 1]])
rot_y2z3 = rot_y2 * rot_z3 # postmultiply because z3 is relative axis
T_23 = sp.diag(rot_y2z3, 1)
T_23[0, 3] = L2

# Frame e: Translation along Y3 by L3, rotation about X3 by -pi/2
T_3e = sp.Matrix([[1, 0, 0, 0],
                [0, 0, 1, L3],
                [0, -1, 0, 0],
                [0, 0, 0, 1]])

T_be = sp.trigsimp(sp.expand(T_b0*T_01*T_12*T_23*T_3e))
    
# Set parameters
T_be = T_be.subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

In [ ]:
# Forward kinematics function
FK = sp.Matrix([T_be[0,3],
                T_be[1,3],
                T_be[2,3],
                sp.atan2(T_be[2,1], T_be[2,2])], # roll
                sp.atan2(-T_be[2,0], sp.sqrt(T_be[0,0]**2 + T_be[1,0]**2)), # oitch
                sp.atan2(T_be[1,0], T_be[0,0])) # yaw

sp.pprint("State dependencies")
print(f"xe = f({FK[0,0].free_symbols})")
print(f"ye = f({FK[1,0].free_symbols})")
print(f"ze = f({FK[2,0].free_symbols})")
print(f"rolle = f({FK[3,0].free_symbols})")
print(f"pitche = f({FK[4,0].free_symbols})")
print(f"yawe = f({FK[5,0].free_symbols})")



State dependencies
xe = f({q_3, q_2})
ye = f({q_1, q_3, q_2})
ze = f({q_1, q_3, q_2})
yawe = f({q_1, q_3, q_2})
rolle = f({q_1, q_3, q_2})
pitche = f({q_1, q_3, q_2})


In [ ]:
# Forward kinematics function
T_Ie = T_Ib * T_be
FK_centralised = sp.Matrix([T_Ie[0,3],
                T_Ie[1,3],
                T_Ie[2,3],
                sp.atan2(-T_Ie[2,0], sp.sqrt(T_Ie[0,0]**2 + T_Ie[1,0]**2)), # roll
                sp.atan2(T_Ie[1,0], T_Ie[0,0]), # 
                sp.atan2(T_Ie[2,1], T_Ie[2,2])])
FK_centralised = sp.trigsimp(sp.expand(FK_centralised))
sp.pprint("State dependencies")
print(f"xe = f({FK_centralised[0,0].free_symbols})")
print(f"ye = f({FK_centralised[1,0].free_symbols})")
print(f"ze = f({FK_centralised[2,0].free_symbols})")
print(f"yawe = f({FK_centralised[3,0].free_symbols})")
print(f"rolle = f({FK_centralised[4,0].free_symbols})")
print(f"pitche = f({FK_centralised[5,0].free_symbols})")

State dependencies
xe = f({pitch, roll, yaw, x_b, q_1, q_3, q_2})
ye = f({pitch, y_b, roll, yaw, q_1, q_3, q_2})
ze = f({pitch, roll, z_b, q_1, q_3, q_2})
yawe = f({pitch, roll, q_1, q_3, q_2})
rolle = f({pitch, roll, yaw, q_1, q_3, q_2})
pitche = f({pitch, roll, q_1, q_3, q_2})


### Analysis of Euler angle singularity
When calculating the end-effector yaw and pitch, there is a division by zero which causes an Euler angle singularity. Therefore, we examine denominators in the expressions for $\theta_{e}$ and $\psi_{e}$. They are rather long and cumbersome expressions on their own due to the many rotations, so we just look at the states they are a function of.

We can simplify a little bit by setting both the body pitch and roll to zero, since the platform needs this for stabilization, and these values are likely to cause or contribute to the singularity. We look again at the independent states. Now, the expression is also much less cumbersome and we can investigate.

In [24]:
# checking out denominator of expression for yawe and pitche
print("Free variables in denominator of yawe and pitche:")
sp.pprint(T_Ie[2,2].free_symbols)
sp.pprint(T_Ie[0,0].free_symbols)

# substitute zero pitch and roll since that is the nominal configuration
T_Ie_22 = T_Ie[2,2].subs([(roll, 0.0), (pitch, 0.0)])
T_Ie_00 = T_Ie[0,0].subs([(roll, 0.0), (pitch, 0.0)])
print("\nWith zero pitch and roll angle:")
print(f"T_Ie = f({T_Ie_22.free_symbols})")
print(f"T_Ie = f({T_Ie_00.free_symbols})")
sp.pprint(T_Ie_22)
sp.pprint(T_Ie_00)

Free variables in denominator of yawe and pitche:
{pitch, q₁, q₂, q₃, roll}
{pitch, q₁, q₂, q₃, roll, yaw}

With zero pitch and roll angle:
T_Ie = f({q_1, q_3, q_2})
T_Ie = f({yaw, q_1, q_3, q_2})
sin(q₁)⋅cos(q₂)⋅cos(q₃) + sin(q₃)⋅cos(q₁)
-(sin(q₁)⋅cos(q₃) + sin(q₃)⋅cos(q₁)⋅cos(q₂))⋅sin(yaw) + sin(q₂)⋅sin(q₃)⋅cos(yaw)


The two lines above mean that no singularities (expression evaluates to 0) exist purely dependent on the yaw.
For end-effector yaw, a singularity exists when $q_{1}$=0 and $q_{3}$=0 (fully stretched sideways), which is intuitive. When $\cos(q_{2})$ is 0, a singularity could exist, but this is irrelevant since the associated angle ($q_{2}$=$\pi /2$) is outside the joint limit.

For the end-effector pitch, a couple of conditions for Euler angle singularities exist. We can evaluate them by looking at when the factors in the expression evaluate to zero. With this method, the following set of singularities can be derived:
$$
\phi_{b} = 0 \land q_{2}=0 \\
\phi_{b} = 0 \land q_{3}=0 \\
q_{1}=0 \land q_{3}=0 \\
\phi_{b} = \pi /2 \land q_{1}=\pi /2 \land q_{3}=\pi /2
$$
Of these, condition IV is also not feasible because $q_{3}=\pi /2$ is outside the joint limit for $q_{3}$

### Controllability
Since the states roll and y_b, and pitch and x_b are coupled, not the entire end-effector configuration space is reachable.

In [20]:
# Jacobian calculation for velocity
J = FK_centralised.jacobian([x, y, z, yaw, pitch, roll, q1, q2, q3])

# Sample values for x, y, z, yaw, pitch, roll, q1, q2, q3
J = J.subs([(x, 0), (y, 1.5), (z, -2.0), (yaw, sp.pi/4.), (pitch, 0.0), (roll, 0.0), (q1, -sp.pi/6), (q2, 0.0), (q3, sp.pi/6)])

# Check for singularity here
sp.pprint(J.evalf())
print(J.shape)
sp.pprint(FK_centralised[5])
J_inv = J.pinv() # use pseudoinverse because J is non-square

# Jacobian calculation for velocity
velocity_reference = sp.Matrix([1, 0, 0, 0, 0, 0]) # 1 m/s in x direction
joint_velocities = J_inv * velocity_reference
sp.pprint(joint_velocities)

⎡1.0   0    0   0.0657609306503489  -0.0673609679265374  -0.0673609679265374   ↪
⎢                                                                              ↪
⎢ 0   1.0   0   -0.374059487247684  -0.0673609679265374  0.0673609679265374    ↪
⎢                                                                              ↪
⎢ 0    0   1.0          0                  0.311                0.218          ↪
⎢                                                                              ↪
⎢ 0    0    0           0                    0                   nan           ↪
⎢                                                                              ↪
⎢ 0    0    0          nan                  nan                  nan           ↪
⎢                                                                              ↪
⎣ 0    0    0           0                   zoo                   0            ↪

↪ -0.0673609679265374  -0.0232701525564402    0  ⎤
↪                                                ⎥
↪ 0.06

NonInvertibleMatrixError: Matrix det == 0; not invertible.